Install dependencies 

In [ ]:
pip install opencv-python matplotlib numpy

Folder structure : 
 i221908_Hamza_Assignment1

- images -> coin_image.jpg, cars.jpeg, Q3.png

- results -> (will be completed in phase 2)


**Libraries**

In [2]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import time
from collections import deque

# **QUESTION 1**

A cc labeling algorithm is technique used to detect and label components/objects in binary image (white black, only 2 pixel values).

we'll be applying the neighbouring wali cheez to detect objects by checking similar bits/pixels ki intensity/color which is 1
now binarization
At first i kept the coins as black and background as white, which led to inconsistent labelling.
when using bfs to find connected comppnenets, woh white spaces ko check kar raha tha instead of black.
Hence it was counting the whole image as 1 component.

bfs function to iterate quickly and lbaleling the correct pixels
- image of a number plate 
- number : HAM-1908
- Will label digits as digits and letters as letters

Why Otsu's Thresholding?
It automatically finds the best threshold value to separate foreground (text) from background.
Connected Component Labeling (CCL) using BFS

In [53]:

def q1_A():
    coin_img = cv.imread("images/coin image.jpeg")
    height,width,channels = coin_img.shape
    print(height,width,channels)
    # convert firdt from bgr to rgb 
    coin_img = cv.cvtColor(coin_img, cv.COLOR_BGR2RGB)

    # ab display the orginal image
    plt.figure()
    plt.imshow(coin_img)
    plt.show()

    # Save the original image
    cv.imwrite('output/original_image_coins.png', coin_img)

    # resizing the image, reducing teh size for easier processing.
    scale_percent = 30  # Adjust percentage as needed
    new_width = int(width * scale_percent / 100)
    new_height = int(height * scale_percent / 100)

    coin_img = cv.resize(coin_img, (new_width, new_height))

    height, width, _ = coin_img.shape

    # convert to grayscale  
    coin_gray = cv.cvtColor(coin_img, cv.COLOR_RGB2GRAY)
    # coin_gray = cv.cvtColor(coin_gray, cv.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(coin_gray, cmap= 'gray')
    plt.show()
    cv.imwrite('output/gray_image_coins.png', coin_gray)

    print(coin_gray)
    print(coin_gray.shape)

    # threshold = 161.9899440342233
    # print(threshold)

    # #161.9899440342233
    # # 172

    _, binary_image = cv.threshold(coin_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    plt.figure()
    plt.imshow(binary_image, cmap='gray')

    cv.imwrite('output/binary_image_coins.png', binary_image)

    start_time = time.perf_counter()

    # need to store label so same size array banoao that will store har pixel ka label
    labels = np.zeros_like(binary_image, dtype=np.int32)

    current_label = 1

    # for 8 neighbour 
    #   (x-1,y-1) (x,y-1) (x+1,y-1)
    #   (x-1,y)   (x,y)   (x+1,y)
    #   (x-1,y+1) (x,y+1) (x+1,y+1)

    directions = [(-1,-1), (0,-1), (1,-1), (-1,0), (1,0), (-1,1), (0,1), (1,1)]
    rows, cols = binary_image.shape


    rows, cols = binary_image.shape

    def bfs(start_i, start_j, label):
        queue = deque([(start_i, start_j)])
        labels[start_i, start_j] = label
        
        while queue:
            i, j = queue.popleft()
            for di, dj in directions:
                ni, nj = i + di, j + dj
                if 0 <= ni < rows and 0 <= nj < cols and binary_image[ni, nj] == 255 and labels[ni, nj] == 0:
                    labels[ni, nj] = label
                    queue.append((ni, nj))
    # assigning labels to pixels/componenets that connected to each other
    # this will also tell joh joh coin ki pixel ramge main ata

    for i in range(rows):
        for j in range(cols):
            if binary_image[i, j] == 255 and labels[i, j] == 0:
                bfs(i, j, current_label)
                current_label += 1  # Increment label for the next component

    end_time = time.perf_counter()
    bfs_time = end_time - start_time
    print(f"Custom BFS CCL Time: {bfs_time:.6f} seconds")


    plt.figure()
    plt.imshow(labels, cmap='nipy_spectral')
    plt.title(f"Connected Components: {current_label - 1}")
    plt.colorbar()
    plt.show()
    cv.imwrite('output/ccl_image_coins.png', labels)

    # Count unique labels (excluding background)
    # unique_labels = np.unique(labels[labels > 0])

    num_components = current_label - 1  

    min_size = 500  # Increase based on your image

    unique_labels = [label for label in np.unique(labels) if np.sum(labels == label) > min_size and label > 0]
    num_coins = len(unique_labels)
    print(f"Number of coins: {num_coins}")

    # Find contours & draw bounding boxes
    contours, _ = cv.findContours(binary_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(coin_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display results
    plt.figure(figsize=(10, 6))
    plt.imshow(cv.cvtColor(coin_img, cv.COLOR_RGB2BGR))
    plt.title(f"Coins Detected: {num_coins}")
    plt.axis('off')
    plt.show()
    cv.imwrite('output/detected_image_coins.png', coin_img)

    ######################################
    # Part 2

    car_img = cv.imread('images/cars.jpeg')
    height,width = car_img.shape[:2]

    print(height,width)

    car_img = cv.cvtColor(car_img, cv.COLOR_BGR2RGB) 
    plt.axis('off')
    plt.imshow(car_img)
    cv.imwrite('output/original_image_cars.png', car_img)
    # convert to grayscale  
    car_gray = cv.cvtColor(car_img, cv.COLOR_RGB2GRAY)
    plt.figure()
    plt.imshow(car_gray, cmap= 'gray')
    plt.show()
    cv.imwrite('output/gray_image_cars.png', car_gray)
    print(car_gray)
    print(car_gray.shape)
    _, binary_image = cv.threshold(car_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Apply distance transform and threshold
    dist_transform = cv.distanceTransform(binary_image, cv.DIST_L2, 5)
    _, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(binary_image, sure_fg)


    def bfs_connected_components(binary_img):
        h, w = binary_img.shape
        visited = np.zeros_like(binary_img, dtype=bool)
        labels = np.zeros_like(binary_img, dtype=np.int32)
        label = 0
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for i in range(h):
            for j in range(w):
                if binary_img[i, j] == 255 and not visited[i, j]:
                    label += 1
                    queue = deque([(i, j)])
                    visited[i, j] = True
                    labels[i, j] = label
                    
                    while queue:
                        x, y = queue.popleft()
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < h and 0 <= ny < w:
                                if binary_img[nx, ny] == 255 and not visited[nx, ny]:
                                    visited[nx, ny] = True
                                    labels[nx, ny] = label
                                    queue.append((nx, ny))
        return label, labels


    start_time = time.time()
    # Get connected components manually
    num_labels, markers = bfs_connected_components(sure_fg)
    end_time = time.time()

    # Calculate execution time
    execution_time = end_time - start_time
    print(f"Execution Time of Custom CCL Algorithm: {execution_time:.6f} seconds")

    # Generate random colors for visualization
    colors = np.random.randint(0, 255, size=(num_labels + 1, 3), dtype=np.uint8)
    label_colored = np.zeros((*markers.shape, 3), dtype=np.uint8)

    for i in range(1, num_labels + 1):
        label_colored[markers == i] = colors[i]

    # Count the number of separate cars
    num_cars = num_labels  # Exclude background label

    # Draw bounding boxes
    contours, _ = cv.findContours(sure_fg, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(car_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display results
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv.cvtColor(car_img, cv.COLOR_RGB2BGR))
    plt.title(f"Cars Detected: {num_cars}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(label_colored)
    plt.title("Separated Components with Colors")
    plt.axis('off')


    cv.imwrite('output/detected_image_cars.png', car_img)

    plt.show()

    print(f"Number of cars: {num_cars}")

    ##############################
    # Part 3


    # Load image
    plate = cv.imread('images/Q3.jpeg')
    height, width, channels = plate.shape
    print(f"Image Dimensions: {height}x{width}x{channels}")

    # Convert to RGB for display
    plate_rgb = cv.cvtColor(plate, cv.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(plate_rgb)
    plt.axis('off')
    plt.show()
    cv.imwrite('output/original_image_plate.png', plate_rgb)


    # Convert to Grayscale
    plate_gray = cv.cvtColor(plate_rgb, cv.COLOR_RGB2GRAY)
    plt.figure()
    plt.imshow(plate_gray, cmap='gray')
    plt.axis('off')
    plt.show()

    cv.imwrite('putput/gray_image_plate.png', plate_gray)

    # Otsu's thresholding (to get binary image)
    _, binary_image = cv.threshold(plate_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Invert binary image so objects are white
    binary_image = cv.bitwise_not(binary_image)

    plt.figure()
    plt.imshow(binary_image, cmap='gray')
    plt.title("Binary Image")
    plt.axis('off')
    plt.show()

    cv.imwrite('output/binary_image_plate.png', binary_image)


    start_time = time.time()
    labels = np.zeros_like(binary_image, dtype=np.int32)  # Store component labels
    current_label = 1  # Start labeling from 1
    directions = [(-1,-1), (0,-1), (1,-1), (-1,0), (1,0), (-1,1), (0,1), (1,1)]  # 8-connected neighbors
    rows, cols = binary_image.shape

    # Random colors for components
    np.random.seed(42)  # For consistent colors
    colors = np.random.randint(0, 255, size=(500, 3), dtype=np.uint8)  # Max 500 labels

    # BFS Function for Labeling
    def bfs(start_i, start_j, label):
        queue = deque([(start_i, start_j)])
        labels[start_i, start_j] = label
        component_pixels = [(start_i, start_j)]  # Store pixels in the component

        while queue:
            i, j = queue.popleft()
            for di, dj in directions:
                ni, nj = i + di, j + dj
                if 0 <= ni < rows and 0 <= nj < cols and binary_image[ni, nj] == 255 and labels[ni, nj] == 0:
                    labels[ni, nj] = label
                    queue.append((ni, nj))
                    component_pixels.append((ni, nj))
        
        return component_pixels


    # Finding Connected Components
    components = []
    for i in range(rows):
        for j in range(cols):
            if binary_image[i, j] == 255 and labels[i, j] == 0:
                component = bfs(i, j, current_label)
                components.append((current_label, component))  # Store label & pixels
                current_label += 1  # Increment label

    end_time = time.time()
    print(f"Time Taken: {end_time - start_time:.2f} seconds")

    output = np.zeros((rows, cols, 3), dtype=np.uint8)


    for label, pixels in components:
        color = colors[label % 500]  # Assign a unique color (mod to avoid overflow)
        for (x, y) in pixels:
            output[x, y] = color  # Apply color

    # Show colored connected components
    plt.figure(figsize=(10, 6))
    plt.imshow(output)
    plt.title(f"Connected Components: {current_label - 1}")
    plt.axis('off')
    plt.show()

    cv.imwrite('output/detected_image_plate.png', output)


    plate_with_boxes = plate_rgb.copy()
    min_size = 500  # Filter out small noise

    for label, pixels in components:
        if len(pixels) > min_size:  # Ignore small noise components
            x_vals, y_vals = zip(*pixels)
            x_min, x_max = min(x_vals), max(x_vals)
            y_min, y_max = min(y_vals), max(y_vals)
            
            # Draw bounding box with the same color as the component
            color = tuple(map(int, colors[label % 500]))  # Convert NumPy array to tuple
            cv.rectangle(plate_with_boxes, (y_min, x_min), (y_max, x_max), color, 2)

    # Show final image with bounding boxes
    plt.figure(figsize=(10, 6))
    plt.imshow(plate_with_boxes)
    plt.title("Plate with Colored Bounding Boxes")
    plt.axis('off')
    plt.show()

    cv.imwrite('output/bounded_image_plate.png', plate_with_boxes)



_______________________________

Part 4

OPENCV's In-built functions for connect components 

- connectedComponents: This function performs connected component labeling and returns the number of connected components and a label image where each pixel is labeled with the index of its corresponding component.
- connectedComponentsWithStats: This function is similar to connectedComponents but also calculates statistics for each connected component, such as bounding box, area, and centroid.
- connectedComponentsWithAlgorithm: This function allows you to specify the algorithm to use for connected component analysis.
- connectedComponentsWithStatsWithAlgorithm: This function combines the functionality of connectedComponentsWithStats and connectedComponentsWithAlgorithm.

Q1 


In [51]:

def q1_b():
    coin_img = cv.imread("images/coin image.jpeg")

    height,width,channels = coin_img.shape
    print(height,width,channels)

    # convert firdt from bgr to rgb 
    coin_img = cv.cvtColor(coin_img, cv.COLOR_BGR2RGB)

    # ab display the orginal image
    plt.figure()
    plt.imshow(coin_img)
    plt.show()

    # resizing the image, reducing teh size for easier processing.
    scale_percent = 30  # Adjust percentage as needed
    new_width = int(width * scale_percent / 100)
    new_height = int(height * scale_percent / 100)

    coin_img = cv.resize(coin_img, (new_width, new_height))


    height, width, _ = coin_img.shape

    # convert to grayscale  
    coin_gray = cv.cvtColor(coin_img, cv.COLOR_RGB2GRAY)
    # coin_gray = cv.cvtColor(coin_gray, cv.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(coin_gray, cmap= 'gray')
    plt.show()

    print('gray scaled image pixel values : ',coin_gray)

    print('size representation : ',coin_gray.shape)


    _, binary_image = cv.threshold(coin_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    plt.figure()
    plt.imshow(binary_image, cmap='gray')

    start_time = time.time()
    num_labels, labels = cv.connectedComponents(binary_image)
    end_time = time.time()

    processing_time_custom = end_time - start_time
    print(f"Inbuilt CCL Processing Time: {processing_time_custom:.6f} seconds")

    plt.figure()
    plt.imshow(labels, cmap='nipy_spectral')
    plt.title(f"Connected Components: {num_labels - 1}")
    plt.colorbar()
    plt.show()

    # Filtering small noise components
    min_size = 500  # Adjust based on image
    unique_labels = [label for label in range(1, num_labels) if np.sum(labels == label) > min_size]
    num_coins = len(unique_labels)

    print(f"Number of coins: {num_coins}")

    # Find contours & draw bounding boxes
    contours, _ = cv.findContours(binary_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(coin_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display detected coins
    plt.figure(figsize=(10, 6))
    plt.imshow(cv.cvtColor(coin_img, cv.COLOR_RGB2BGR))
    plt.title(f"Coins Detected: {num_coins}")
    plt.axis('off')
    plt.show()

    ###### question 1 part 4 part 2

    car_img = cv.imread('images/cars.jpeg')
    height,width = car_img.shape[:2]

    print(height,width)

    car_img = cv.cvtColor(car_img, cv.COLOR_BGR2RGB) 
    plt.axis('off')
    plt.imshow(car_img)

    # convert to grayscale  
    car_gray = cv.cvtColor(car_img, cv.COLOR_RGB2GRAY)

    print(car_gray)
    print(car_gray.shape)


    threshold_value = 180  # Adjust as needed (try 80, 90, etc.)
    _, binary_image = cv.threshold(car_gray, threshold_value, 255, cv.THRESH_BINARY_INV)

    kernel = np.ones((5,5), np.uint8)
    binary_image = cv.morphologyEx(binary_image, cv.MORPH_CLOSE, kernel)

    plt.figure()
    plt.axis('off')
    plt.imshow(binary_image, cmap='gray')
    # Apply distance transform
    dist_transform = cv.distanceTransform(binary_image, cv.DIST_L2, 5)
    dist_transform = cv.normalize(dist_transform, None, 0, 1.0, cv.NORM_MINMAX)

    # Threshold to obtain sure foreground (car centers)
    _, sure_fg = cv.threshold(dist_transform, 0.5, 1.0, cv.THRESH_BINARY)
    sure_fg = np.uint8(sure_fg * 255)

    # Find sure background
    sure_bg = cv.dilate(binary_image, kernel, iterations=3)

    # Get unknown region
    unknown = cv.subtract(sure_bg, sure_fg)

    # Label markers for watershed
    start_time_cars = time.time()


    num_labels, markers = cv.connectedComponents(sure_fg)

    end_time_cars = time.time()

    processing_time_custom_cars = end_time_cars - start_time_cars
    print(f"Inbuilt CCL Processing Time: {processing_time_custom_cars:.6f} seconds")


    markers = markers + 1  # Ensure background is not zero
    markers[unknown == 255] = 0  # Mark unknown regions as 0

    # Apply watershed algorithm
    car_img_bgr = cv.cvtColor(car_img, cv.COLOR_RGB2BGR)
    cv.watershed(car_img_bgr, markers)
    car_img[markers == -1] = [255, 0, 0]  # Mark watershed boundaries in red

    start_time_cars1 = time.time()

    # Apply OpenCV's connected component analysis
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(sure_fg, connectivity=8)

    num_labels, markers = cv.connectedComponents(sure_fg)

    end_time_cars1 = time.time()
    execution_time = end_time_cars1 - start_time_cars1
    print(f"Inbuilt CCL Processing Time: {execution_time:.6f} seconds")

    # Filter components based on size
    min_size = 600
    valid_labels = [i for i in range(1, num_labels) if stats[i, cv.CC_STAT_AREA] > min_size]
    num_cars = len(valid_labels)

    # Generate random colors for labels
    colors = np.random.randint(0, 255, size=(num_labels, 3), dtype=np.uint8)
    label_colored = np.zeros((*labels.shape, 3), dtype=np.uint8)

    for i in valid_labels:
        label_colored[labels == i] = colors[i]

    # Draw bounding boxes around detected cars
    for i in valid_labels:
        x, y, w, h, area = stats[i]
        cv.rectangle(car_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display results
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv.cvtColor(car_img, cv.COLOR_RGB2BGR))
    plt.title(f"Cars Detected: {num_cars}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(label_colored)
    plt.title("Connected Components with Colors")
    plt.axis('off')

    plt.show()

    print(f"Number of cars: {num_cars}")

    ##### part 3


    # Load image
    plate = cv.imread('images/Q3.jpeg')
    height, width, channels = plate.shape
    print(f"Image Dimensions: {height}x{width}x{channels}")


    # Convert to RGB for display
    plate_rgb = cv.cvtColor(plate, cv.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(plate_rgb)
    plt.axis('off')
    plt.show()


    # Convert to Grayscale
    plate_gray = cv.cvtColor(plate_rgb, cv.COLOR_RGB2GRAY)
    plt.figure()
    plt.imshow(plate_gray, cmap='gray')
    plt.axis('off')
    plt.show()


    # Otsu's thresholding (to get binary image)
    _, binary_image = cv.threshold(plate_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Invert binary image so objects are white
    binary_image = cv.bitwise_not(binary_image)

    plt.figure()
    plt.imshow(binary_image, cmap='gray')
    plt.title("Binary Image")
    plt.axis('off')
    plt.show()




    # Connected Component Analysis using OpenCV
    start_time_plate = time.perf_counter()
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(binary_image, connectivity=8)
    end_time_plate = time.perf_counter()

    processing_time_custom_plate = end_time_plate - start_time_plate
    print(f"inbuilt CCL Processing Time: {processing_time_custom_plate:.6f} seconds")


    # Create an output image with colors
    output = np.zeros((binary_image.shape[0], binary_image.shape[1], 3), dtype=np.uint8)

    # Assign random colors to each component (excluding the background)
    colors = np.random.randint(0, 255, size=(num_labels, 3), dtype=np.uint8)

    for label in range(1, num_labels):  # Start from 1 to ignore background
        output[labels == label] = colors[label]

    # Show colored components
    plt.figure(figsize=(10, 6))
    plt.imshow(output)
    plt.title("Connected Components with Different Colors")
    plt.axis('off')
    plt.show()

    # Draw bounding boxes for valid components
    plate_with_boxes = plate_rgb.copy()
    min_size = 500  # Filter small noise components

    for label in range(1, num_labels):  # Ignore background
        x, y, w, h, area = stats[label]
        if area > min_size:
            cv.rectangle(plate_with_boxes, (x, y), (x + w, y + h), (int(colors[label][0]), int(colors[label][1]), int(colors[label][2])), 2)  # Assign the same color

    # Show final image with bounding boxes
    plt.figure(figsize=(10, 6))
    plt.imshow(plate_with_boxes)
    plt.title("Plate with Colored Bounding Boxes")
    plt.axis('off')
    plt.show()


# **Question 2**

Part 1

In [ ]:
def q2():

    selfie = cv.imread('images\Q2A.png')
    plt.imshow(cv.cvtColor(selfie, cv.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    print(selfie.shape)


    def encode_text(image, text, bits):
        text += "####"  # Delimiter to mark the end of the message
        binary_text = ''.join(format(ord(char), '08b') for char in text)
        
        img = image.copy()
        data_idx = 0
        # bits = 2  # Always using 2 bits
        max_bytes = (img.shape[0] * img.shape[1] * img.shape[2] * bits) // 8
        
        if len(binary_text) > max_bytes * 8:
            raise ValueError("Message too long to encode in this image")

        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                for k in range(img.shape[2]):  
                    if data_idx < len(binary_text):
                        img[i, j, k] = (img[i, j, k] & ~(2**bits - 1)) | int(binary_text[data_idx:data_idx+bits], 2)
                        data_idx += bits
                    else:
                        return img
        return img

    # Function to decode text from an image
    def decode_text(image, bits=2):
        binary_text = ""
        
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):
                    binary_text += format(image[i, j, k] & (2**bits - 1), f'0{bits}b')

        chars = [binary_text[i:i+8] for i in range(0, len(binary_text), 8)]
        message = ''.join(chr(int(c, 2)) for c in chars)

        return message.split("####")[0]  # Extract message before the delimiter

    message = "Wayne gretski"
    encoded_image = encode_text(selfie, message, bits=2)
    decoded_message = decode_text(encoded_image, bits=2)

    plt.subplot(1,2,2)
    plt.imshow(cv.cvtColor(encoded_image, cv.COLOR_BGR2RGB))
    plt.title("Encoded Image")
    plt.axis("off")
    plt.show()


    print("decoded message : ", decoded_message)


    #####################################################
    #PART 2 OF QUESTION 2
    def encode_text(image, text, bits):
        text += "####"  # Delimiter to mark the end of the message
        binary_text = ''.join(format(ord(char), '08b') for char in text)
        
        img = image.copy()
        data_idx = 0
        # bits = 2  # Always using 2 bits
        max_bytes = (img.shape[0] * img.shape[1] * img.shape[2] * bits) // 8
        
        if len(binary_text) > max_bytes * 8:
            raise ValueError("Message too long to encode in this image")

        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                for k in range(img.shape[2]):  
                    if data_idx < len(binary_text):
                        img[i, j, k] = (img[i, j, k] & ~(2**bits - 1)) | int(binary_text[data_idx:data_idx+bits], 2)
                        data_idx += bits
                    else:
                        return img
        return img

    # Function to decode text from an image
    def decode_text(image, bits=2):
        binary_text = ""
        
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):
                    binary_text += format(image[i, j, k] & (2**bits - 1), f'0{bits}b')

        chars = [binary_text[i:i+8] for i in range(0, len(binary_text), 8)]
        message = ''.join(chr(int(c, 2)) for c in chars)

        return message.split("####")[0]  # Extract message before the delimiter


    image_path = input("Enter the path to the image: ")
    image = cv.imread(image_path)

    if image is None:
        print("Error: Could not load image.")
    else:
        print("Image Loaded Successfully.")


    message_to_hide = input("Enter the secret message to hide: ")

        
    encoded_image = encode_text(image, message_to_hide, bits = 2)
    print(f"Secret message successfully hidden.")

    extracted_message = decode_text(encoded_image)
    print(f"Extracted Message: {extracted_message}")

    #########################################
    # part 3 of question 2 
    # Function to hide the secret image inside the cover image using bitwise operations
    def hide_image(cover_img, secret_img):
        # Resize secret image to match cover image dimensions
        secret_img = cv.resize(secret_img, (cover_img.shape[1], cover_img.shape[0]))

        # Ensure both images have 3 channels (RGB)
        if len(cover_img.shape) == 2:
            cover_img = cv.cvtColor(cover_img, cv.COLOR_GRAY2BGR)
        if len(secret_img.shape) == 2:
            secret_img = cv.cvtColor(secret_img, cv.COLOR_GRAY2BGR)

        # Step 1: Clear last 2 bits of cover image by shifting right then left
        cleared_cover = (cover_img >> 2) << 2  

        # Step 2: Store the top 2 bits of secret image in last 2 bits of cover image
        encoded_img = cleared_cover | (secret_img >> 6)  

        return encoded_img

    def extract_image(encoded_img):
        # Extract the hidden image by shifting back to original position
        extracted_img = (encoded_img & 0b00000011) << 6  

        return extracted_img

    cover_image_path = input("Enter the path to the cover image: ")
    secret_image_path = input("Enter the path to the secret image: ")

    # Load the images
    cover_image = cv.imread(cover_image_path)
    secret_image = cv.imread(secret_image_path)

    # Check if images are loaded properly
    if cover_image is None or secret_image is None:
        print("Error: Could not load one or both images.")
    else:
        print("Images Loaded Successfully.")

        # Encode the secret image inside the cover image
        encoded_image = hide_image(cover_image, secret_image)

        # Display the original cover image
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(cv.cvtColor(cover_image, cv.COLOR_BGR2RGB))
        plt.title("Cover Image")
        plt.axis("off")

        # Display the encoded image
        plt.subplot(1, 3, 2)
        plt.imshow(cv.cvtColor(encoded_image, cv.COLOR_BGR2RGB))
        plt.title("Encoded Cover Image")
        plt.axis("off")

        # Decode and display the extracted secret image
        extracted_secret_image = extract_image(encoded_image)

        plt.subplot(1, 3, 3)
        plt.imshow(cv.cvtColor(extracted_secret_image, cv.COLOR_BGR2RGB))
        plt.title("Extracted Secret Image")
        plt.axis("off")

        plt.show()

    ##################################
    ## part 4

    # Approach:
    # Use the upper 4 bits of the first image (so it's mostly visible).
    # Use the lower 4 bits of the second image (so it's hidden but can be extracted).
    # Ensure bright, contrasting colors for proper testing.
    # Display the final superimposed image.

    # Function to superimpose two images using 4 bits each
    def superimpose_images(img1, img2):
        # Resize second image to match first image dimensions
        img2 = cv.resize(img2, (img1.shape[1], img1.shape[0]))

        # Ensure both images are 3-channel (RGB)
        if len(img1.shape) == 2:
            img1 = cv.cvtColor(img1, cv.COLOR_GRAY2BGR)
        if len(img2.shape) == 2:
            img2 = cv.cvtColor(img2, cv.COLOR_GRAY2BGR)

        # Step 1: Keep upper 4 bits of first image (by shifting right then left)
        img1_upper = (img1 >> 4) << 4

        # Step 2: Keep lower 4 bits of second image (by masking)
        img2_lower = img2 >> 4

        # Step 3: Combine both images
        superimposed_img = img1_upper | img2_lower

        return superimposed_img

    # Take user input for two image paths
    image1_path = 'images\Q2A.png'
    image2_path = 'images\Q2C.jpeg'

    # Load the images
    image1 = cv.imread(image1_path)
    image2 = cv.imread(image2_path)

    # Check if images are loaded properly
    if image1 is None or image2 is None:
        print("Error: Could not load one or both images.")
    else:
        print("Images Loaded Successfully.")

        # Superimpose the images
        superimposed_image = superimpose_images(image1, image2)

        # Display the images
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(cv.cvtColor(image1, cv.COLOR_BGR2RGB))
        plt.title("First Image (Main)")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow(cv.cvtColor(image2, cv.COLOR_BGR2RGB))
        plt.title("Second Image (Hidden)")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(cv.cvtColor(superimposed_image, cv.COLOR_BGR2RGB))
        plt.title("Superimposed Image")
        plt.axis("off")

        plt.show()


In [ ]:
while True:
        print("\n=== Assignment Menu ===")
        print("1. Q1 (Parts 1-3)")
        print("2. Q1 (Part 4)")
        print("3. Q2")
        print("4. Exit")

        choice = input("Select an option: ")

        if choice == '1':
            q1_A()  # Calls Q1 Parts 1-3
        elif choice == '2':
            q1_b()  # Calls Q1 Part 4
        elif choice == '3':
            q2()  # Calls Q2
        elif choice == '4':
            print("Exiting program...")
            break
        else:
            print("Invalid option! Try again.")

References : 

1. https://chatgpt.com/
2. https://www.freepik.com/ai/image-generator
3. Selfie from @Hamzaasad.d on IG
